
Segmentación de Clientes y Predicción de Compra
Segmentación de Clientes y Predicción de Comportamiento de Compra

Contexto

Una plataforma de comercio electrónico en crecimiento, quiere mejorar su estrategia de marketing mediante el análisis del comportamiento de sus clientes. El objetivo es segmentar a los clientes en grupos con características similares y predecir si un cliente realizará una compra en el próximo mes.

Para ello, utilizarás dos algoritmos de Machine Learning:

K-Means para agrupar a los clientes en segmentos basados en su historial de compras.

Regresión Logística para predecir si un cliente comprará en el próximo mes.

Objetivo del Proyecto

El objetivo es desarrollar un modelo de segmentación y predicción de clientes que permita a la empresa tomar mejores decisiones de marketing, ofreciendo promociones personalizadas a cada segmento de clientes.

Tareas principales:

Generar un conjunto de datos sintético con información de clientes.

Implementar un modelo de segmentación con K-Means para agrupar clientes.

Entrenar un modelo de clasificación con Regresión Logística para predecir compras futuras.

Evaluar la precisión del modelo y visualizar la matriz de confusión.

Escribir pruebas unitarias para verificar el correcto funcionamiento del modelo.

Datos Disponibles

Debes generar un conjunto de datos ficticio con 500 clientes, donde cada cliente tiene las siguientes características:

total_spent (float): Monto total gastado en la tienda (entre $100 y $5000).

total_purchases (int): Número total de compras realizadas (entre 1 y 100).

purchase_frequency (float): Promedio de días entre compras (entre 1 y 30 días).

will_buy_next_month (int): Variable binaria (1 = comprará, 0 = no comprará), con una probabilidad del 30% de comprar y 70% de no comprar.

Fases del Desarrollo

1. Generación de Datos

Crea un conjunto de datos sintético que cumpla con las características anteriores. Para esto, usa numpy y pandas para generar valores aleatorios realistas.

2. Implementar el Modelo de Segmentación (K-Means)

Usa el algoritmo K-Means para agrupar a los clientes en tres segmentos en función de:

total_spent (gasto total).

total_purchases (número de compras).

purchase_frequency (frecuencia de compra).

Después de aplicar K-Means, agrega una nueva columna llamada customer_segment que indique el segmento asignado a cada cliente.

3. Implementar el Modelo de Predicción (Regresión Logística)

Después de la segmentación, usa Regresión Logística para predecir si un cliente realizará una compra en el próximo mes.

Usa como variables de entrada (X) las características originales más la columna de segmentación.

Usa will_buy_next_month como variable objetivo (y).

Divide los datos en conjunto de entrenamiento y prueba (80%-20%).

Entrena el modelo de Regresión Logística y evalúa su desempeño.

4. Evaluación del Modelo

Después del entrenamiento, mide el desempeño del modelo usando:

Precisión (accuracy_score).

Matriz de confusión (confusion_matrix).

La precisión del modelo debe ser al menos 60% para considerar que tiene buen rendimiento.



In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import unittest
import sys

class CustomerSegmentationModel:
    def __init__(self, input_data):
        if isinstance(input_data, (int, float)):
            self.n_customers = int(input_data)
            self.data = None
        elif isinstance(input_data, pd.DataFrame):
            self.data = input_data.copy()
            self.n_customers = len(self.data)
        else:
            raise ValueError("El parámetro debe ser un número o un DataFrame.")
        
        self.kmeans = None
        self.logreg = None
        self.y_test = None
        self.y_pred = None
        self.accuracy = None

    def generate_data(self):
        np.random.seed(42)
        n = self.n_customers

        self.data = pd.DataFrame({
            'total_spent': np.random.uniform(100, 5000, n),
            'total_purchases': np.random.randint(1, 100, n),
            'purchase_frequency': np.random.uniform(1, 30, n),
            'will_buy_next_month': np.random.choice([0, 1], size=n, p=[0.7, 0.3])
        })

    def segment_customers(self, n_clusters=3):
        features = self.data[['total_spent', 'total_purchases', 'purchase_frequency']]
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=5, max_iter=100)
        self.data['customer_segment'] = self.kmeans.fit_predict(features)

    def train_model(self):
        X = self.data[['total_spent', 'total_purchases', 'purchase_frequency', 'customer_segment']]
        y = self.data['will_buy_next_month']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        self.logreg = LogisticRegression(max_iter=200)
        self.logreg.fit(X_train, y_train)

        self.y_test = y_test
        self.y_pred = self.logreg.predict(X_test)
        self.accuracy = accuracy_score(y_test, self.y_pred)
        return self.accuracy

    def get_accuracy(self):
        return self.accuracy

    def get_confusion_matrix(self):
        return confusion_matrix(self.y_test, self.y_pred)


# ------------------ PRUEBAS UNITARIAS ------------------

class TestCustomerSegmentationModel(unittest.TestCase):
    def setUp(self):
        self.model = CustomerSegmentationModel(200)
        self.model.generate_data()
        self.model.segment_customers()
        self.model.train_model()

    def test_data_generated(self):
        self.assertIsNotNone(self.model.data)
        self.assertEqual(len(self.model.data), 200)

    def test_customer_segment_column(self):
        self.assertIn("customer_segment", self.model.data.columns)

    def test_model_training(self):
        accuracy = self.model.get_accuracy()
        self.assertGreaterEqual(accuracy, 0.5)

    def test_confusion_matrix_shape(self):
        cm = self.model.get_confusion_matrix()
        self.assertEqual(cm.shape, (2, 2))


# ------------------ EJECUCIÓN ------------------

if __name__ == "__main__":
    if "test" in sys.argv:
        unittest.main(argv=[''], exit=False)
    else:
        model = CustomerSegmentationModel(200)
        model.generate_data()
        model.segment_customers()
        acc = model.train_model()
        print("Precisión:", acc)
        print("Matriz de confusión:")
        print(model.get_confusion_matrix())


Precisión: 0.75
Matriz de confusión:
[[30  0]
 [10  0]]


C:\Users\Acer\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
